In [5]:
from __future__ import print_function
import torch
import torchvision

m_train = torchvision.datasets.MNIST(root="MNIST_data/",train=True,transform=torchvision.transforms.ToTensor(),download=True)
m_test = torchvision.datasets.MNIST(root="MNIST_data/",train=False,transform=torchvision.transforms.ToTensor(),download=True)

data_loader = torch.utils.data.DataLoader(m_train,batch_size=1000, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(m_test)
device = torch.device("cuda:0")

class MnistModel(torch.nn.Module):
  def __init__(self):
    super(MnistModel, self).__init__()

    self.conv1 = torch.nn.Conv2d(1, 32, 5, padding=2)
    self.conv2 = torch.nn.Conv2d(32,64,5,padding=2)

    self.fc1 = torch.nn.Linear(64*7*7,1024)
    self.fc2 = torch.nn.Linear(1024,10)

  def forward(self, x):
    x = torch.nn.functional.max_pool2d(torch.nn.functional.relu(self.conv1(x)),2)
    x = torch.nn.functional.max_pool2d(torch.nn.functional.relu(self.conv2(x)),2)
    x = x.view(-1,64*7*7)
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.dropout(x, training=self.training)
    x = self.fc2(x)
    return torch.nn.functional.log_softmax(x)


model = MnistModel().to(device)

total_batch = len (data_loader)
epoch = 10
model_train=model.train()
for epo in range(epoch):
  total_cost = 0
  for x, y in data_loader:
    x = x.to(device)
    y = y.to(device)
    opt = torch.optim.Adadelta(model_train.parameters(), lr=1.0)
    opt.zero_grad()
    output = model_train(x)
    loss = torch.nn.functional.nll_loss(output, y)
    loss.backward()
    opt.step()

    total_cost += loss

  avg_cost = total_cost / total_batch
  print("에폭: %d" % (epo+1), "코스트 = %.9f" % (avg_cost))

test_loss = 0
correct = 0

model_test=model.eval()
with torch.no_grad():
  for x_test,y_test in test_loader:
    x_test = x_test.to(device)
    y_test = y_test.to(device)
    output = model_test(x_test)
    test_loss += torch.nn.functional.nll_loss(output, y_test, reduction='sum').item()
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.eq(y_test.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print("정확도 : ", 100.0 * correct / len(test_loader.dataset),'%')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


에폭: 1 코스트 = 0.882410824
에폭: 2 코스트 = 0.131628677
에폭: 3 코스트 = 0.076485597
에폭: 4 코스트 = 0.056306198
에폭: 5 코스트 = 0.047017135
에폭: 6 코스트 = 0.037850585
에폭: 7 코스트 = 0.033070933
에폭: 8 코스트 = 0.028149327
에폭: 9 코스트 = 0.025919082
에폭: 10 코스트 = 0.021788221
정확도 :  99.29 %
